In [23]:
# split graph into specific nodes per subject.

import ansible_vault
import json
import pathlib
import rdflib

graph_in = rdflib.Graph()
graph_in.parse(pathlib.Path.cwd().parents[0] / "example" / "example.json")

ont = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/")
res = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/")

graph_in.bind("ont", ont)
graph_in.bind("res", res)

subject_matter = dict()

with open(pathlib.Path.cwd().parents[0] / "example" / "keys.json") as keys:
    keys = json.load(keys)

openstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.OpenStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in openstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        fragment.parse(data=o)
        for a, b, c in fragment.triples((None, None, None)):
            subject_matter[s] = a

closedstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.ClosedStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in closedstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        unencruptr = ansible_vault.Vault(keys[str(s)]).load_raw(o).decode()
        fragment.parse(data=unencruptr)
        for a, b, c in fragment.triples((None, None, None)):
            subject_matter[s] = a

for s, p, o in graph_in.triples((None, None, None)):
    target = pathlib.Path(subject_matter[s]).stem
    target_file = pathlib.Path.cwd() / f"{target}.json"

    fragment = rdflib.Graph()
    if target_file.exists():
        fragment.parse(target_file)
    fragment.add((s, p, o))
    fragment.serialize(destination=target_file, format="json-ld")

print("all done.")


all done.
